In [1]:
import numpy as np
import pywt
import pandas as pd
import librosa

USE_WAVELET = None

NAMES = ['LL','LP','RP','RR']

FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]

# DENOISE FUNCTION
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='haar', level=1):    
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    ret=pywt.waverec(coeff, wavelet, mode='per')
    
    return ret

def spectrogram_from_eeg(eeg, spec_width=256, n_mels=128):    
    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((n_mels,spec_width,4),dtype='float32')

    signals = []
    for k in range(4):
        COLS = FEATS[k]
        
        for kk in range(4):
            print(k, kk, COLS[kk], COLS)
            # COMPUTE PAIR DIFFERENCES
            x = eeg[COLS[kk]].values - eeg[COLS[kk+1]].values

            # FILL NANS
            m = np.nanmean(x)
            if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
            else: x[:] = 0

            # DENOISE
            if USE_WAVELET:
                x = denoise(x, wavelet=USE_WAVELET)
            signals.append(x)

            # RAW SPECTROGRAM
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//spec_width, 
                  n_fft=1024, n_mels=n_mels, fmin=0, fmax=20, win_length=n_mels)

            print(mel_spec.shape)
            
            # LOG TRANSFORM
            width = (mel_spec.shape[1]//32)*32
            print(width)
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]
            print(mel_spec_db.shape)
            # STANDARDIZE TO -1 TO 1
            mel_spec_db = (mel_spec_db+40)/40 
            img[:,:,k] += mel_spec_db
                
        # AVERAGE THE 4 MONTAGE DIFFERENCES
        img[:,:,k] /= 4.0 # (Hz, time)
        
        
    return img

In [4]:
feats = ['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz',
       'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG']

eeg = pd.read_parquet("/home/hiramatsu/kaggle/hms-harmful-brain-activity-classification/data/train_eegs/568657.parquet")
eeg = eeg[:10000]
data = np.zeros((10_000,len(feats)))

for j,col in enumerate(feats):
    
    # FILL NAN
    x = eeg.loc[:, col].values.astype("float32")
    m = np.nanmean(x)
    if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
    else: x[:] = 0
        
    data[:,j] = x

spec_eeg = spectrogram_from_eeg(eeg, 512, 128)
spec_eeg.shape

0 0 Fp1 ['Fp1', 'F7', 'T3', 'T5', 'O1']
(128, 527)
512
(128, 512)
0 1 F7 ['Fp1', 'F7', 'T3', 'T5', 'O1']
(128, 527)
512
(128, 512)
0 2 T3 ['Fp1', 'F7', 'T3', 'T5', 'O1']
(128, 527)
512
(128, 512)
0 3 T5 ['Fp1', 'F7', 'T3', 'T5', 'O1']
(128, 527)
512
(128, 512)
1 0 Fp1 ['Fp1', 'F3', 'C3', 'P3', 'O1']
(128, 527)
512
(128, 512)
1 1 F3 ['Fp1', 'F3', 'C3', 'P3', 'O1']
(128, 527)
512
(128, 512)
1 2 C3 ['Fp1', 'F3', 'C3', 'P3', 'O1']
(128, 527)
512
(128, 512)
1 3 P3 ['Fp1', 'F3', 'C3', 'P3', 'O1']
(128, 527)
512
(128, 512)
2 0 Fp2 ['Fp2', 'F8', 'T4', 'T6', 'O2']
(128, 527)
512
(128, 512)
2 1 F8 ['Fp2', 'F8', 'T4', 'T6', 'O2']
(128, 527)
512
(128, 512)
2 2 T4 ['Fp2', 'F8', 'T4', 'T6', 'O2']
(128, 527)
512
(128, 512)
2 3 T6 ['Fp2', 'F8', 'T4', 'T6', 'O2']
(128, 527)
512
(128, 512)
3 0 Fp2 ['Fp2', 'F4', 'C4', 'P4', 'O2']
(128, 527)
512
(128, 512)
3 1 F4 ['Fp2', 'F4', 'C4', 'P4', 'O2']
(128, 527)
512
(128, 512)
3 2 C4 ['Fp2', 'F4', 'C4', 'P4', 'O2']
(128, 527)
512
(128, 512)
3 3 P4 ['Fp2', 'F4', 

(128, 512, 4)

In [3]:
spec = pd.read_parquet("/home/hiramatsu/kaggle/hms-harmful-brain-activity-classification/data/test_spectrograms/853520.parquet")
spec

,time,LL_0.59,LL_0.78,LL_0.98,LL_1.17,LL_1.37,LL_1.56,LL_1.76,LL_1.95,LL_2.15,...,RP_18.16,RP_18.36,RP_18.55,RP_18.75,RP_18.95,RP_19.14,RP_19.34,RP_19.53,RP_19.73,RP_19.92
0,1,14.910000,17.110001,11.660000,11.73,6.08,4.54,4.31,3.38,2.05,...,0.07,0.06,0.05,0.06,0.05,0.05,0.06,0.05,0.04,0.05
1,3,11.130000,10.950000,10.770000,5.07,4.03,3.24,3.61,2.98,1.54,...,0.05,0.04,0.04,0.04,0.04,0.04,0.03,0.03,0.03,0.02
2,5,10.880000,10.570000,8.790000,5.33,2.44,1.48,1.83,0.99,0.89,...,0.04,0.04,0.04,0.03,0.03,0.04,0.04,0.05,0.06,0.06
3,7,19.450001,18.200001,17.719999,13.38,4.17,1.88,1.84,1.22,1.27,...,0.03,0.03,0.05,0.08,0.07,0.07,0.08,0.03,0.03,0.03
4,9,21.650000,22.530001,23.160000,17.00,7.19,3.89,3.65,2.72,2.35,...,0.04,0.04,0.05,0.05,0.06,0.05,0.05,0.05,0.04,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,591,15.580000,18.209999,14.020000,15.96,4.36,4.98,2.68,2.22,2.03,...,0.48,0.59,0.59,0.73,0.44,0.41,0.56,0.60,0.61,0.60
296,593,17.209999,20.219999,20.889999,17.16,9.15,4.14,2.49,2.71,1.60,...,0.26,0.37,0.41,0.36,0.48,0.36,0.39,0.46,0.34,0.32
297,595,9.610000,13.320000,9.190000,11.50,8.11,5.53,5.57,3.69,3.19,...,0.58,0.37,0.17,0.14,0.13,0.30,0.36,0.39,0.56,0.29
298,597,8.430000,11.840000,13.640000,10.56,8.63,5.80,2.98,1.48,0.96,...,0.54,0.22,0.17,0.16,0.11,0.38,0.45,0.45,0.45,0.34


In [5]:
eeg_img = np.zeros((128, 512, 4))
eeg_img = [eeg_img[:, :, i:i+1] for i in range(4)]
eeg_img = np.concatenate(eeg_img, 0)
eeg_img.shape

(512, 512, 1)

In [6]:
[0]*5

[0, 0, 0, 0, 0]